In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import torch
import clip
from PIL import Image
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from torch.nn.utils.rnn import pad_sequence
from torch.nn import functional as F
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from tensorflow import keras
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
clip.available_models()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model, CliPPreprocess = clip.load('ViT-B/32')

In [ ]:
csv_path = '/content/drive/MyDrive/results.csv'
dataset = pd.read_csv(csv_path, delimiter = '|')
dataset.columns = ['image_name', 'comment_number', 'comment']
dataset['comment_number'] = dataset['comment_number'].str.strip()
dataset = dataset.drop([19995,19996,19997,19998,19999])
dataset['comment_number'] = pd.to_numeric(dataset['comment_number'])
dataset['comment'] = dataset['comment'].str.strip()

captions = dataset['comment'].tolist()

words = []
for caption in captions:
  words.extend(caption.split())
words = [word.lower() for word in words]

from collections import Counter
word_counts = Counter(words)

vocab = []
for word,freq in word_counts.items():
  if freq >= 3:
    vocab.append(word)
vocab = ['<PAD>', '<UNK>','<START>','<END>'] + vocab

word_to_index = {word:index for index,word in enumerate(vocab)}
index_to_word = {index:word for index,word in enumerate(vocab)}

def caption_to_indices(captions, word_to_index):
    # iterate over each caption
    captions_numerical = []
    for caption in captions:
        caption_indices = []
        caption_indices.append(word_to_index['<START>'])
        caption_indices.extend([word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in caption.lower().split()])
        caption_indices.append(word_to_index['<END>'])
        captions_numerical.append(caption_indices)
    return captions_numerical

In [ ]:
import numpy as np

CLIPOutputs_path = '/content/drive/MyDrive/image_representations.npy'
image_representations = np.load(CLIPOutputs_path, allow_pickle = True)

padded_captions_path = '/content/drive/MyDrive/Flickr30kCaptions.npy'
padded_captions = np.load(padded_captions_path, allow_pickle = True)

tensors = [t.squeeze() for t in image_representations]
image_representations = torch.stack(tensors)

In [ ]:
padded_captions = tf.reshape(padded_captions, [31782,5,84])

In [ ]:
import zipfile
import io
import numpy as np
from PIL import Image

zip_path = '/content/drive/MyDrive/flickr30k_images.zip'

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    image_files = [name for name in zip_ref.namelist() if name.endswith('.jpg')]

    # Create an empty NumPy array to store the images
    images = np.empty(len(image_files), dtype=object)

    # Loop over the image file names
    for i, file_name in enumerate(image_files):
        # Read the image data from the zip file
        with zip_ref.open(file_name) as image_file:
            # Open the image using PIL
            image = Image.open(io.BytesIO(image_file.read()))

            # Store the processed image in the array
            images[i] = image


In [ ]:
# Remove image 4000
images = np.delete(images, 3999, axis=0)

In [ ]:
representations = []
i=0
# Process each image and generate representations
for image in images:
    # Preprocess the image using the preprocess function from clip.load
    preprocessed_image = CliPPreprocess(image).unsqueeze(0).to('cuda')
    # Generate the image representation
    with torch.no_grad():
        representation = model.encode_image(preprocessed_image).float()

    # Append the representation to the list
    representations.append(representation)

# Convert the list of representations to a NumPy array
representations_np = np.array(representations)

# Define the path to save the representations
representations_path = '/content/drive/MyDrive/CLIP_outputs.npy'  # Modify as desired

# Save the representations as a .npy file
np.save(representations_path, representations_np)

In [ ]:
numerical_captions = caption_to_indices(captions,word_to_index)

numerical_captions_tensors = [torch.tensor(caption) for caption in numerical_captions]
padded_captions = pad_sequence(numerical_captions_tensors, batch_first = True)

np.save('/content/drive/MyDrive/Flickr30kCaptions.npy', padded_captions)

#important variables are now padded_captions (tensor) and representations_np (numpy array)